In [1]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '1'

import logging
import numexpr as ne
import numpy as np
import torch
import datetime
from ddopai.envs.pricing.dynamic import DynamicPricingEnv
from ddopai.envs.pricing.dynamic_inventory import DynamicPricingInvEnv
from ddopai.envs.actionprocessors import ClipAction, RoundAction

from ddopai.experiments.experiment_functions_online import run_experiment
from ddopai.experiments.meta_experiment_functions import *
import requests
import yaml
import re
import pandas as pd
import wandb
from copy import deepcopy
import warnings
import gc
from mushroom_rl import core 
import pickle

In [2]:
logging_level = logging.INFO
logging.basicConfig(level=logging_level)

ne.set_num_threads(1)
torch.backends.cudnn.enabled = False
torch.set_num_threads(1)

set_warnings(logging.INFO) # turn off warnings for any level higher or equal to the input level
LIBRARIES_TO_TRACK = ["ddopai", "mushroom_rl"]
PROJECT_NAME = "pricing_cMDP_test"

ENVCLASS = DynamicPricingEnv
RESULTS_DIR = "results"
def get_ENVCLASS(class_name):
    if class_name == "DynamicPricingEnv":
        return DynamicPricingEnv
    elif class_name == "DynamicPricingInvEnv":
        return DynamicPricingInvEnv
    else:
        raise ValueError(f"Unknown class name {class_name}")

# Experiment preparations
## Set-up WandB
### Init WandB

In [3]:
project_name = "pricing_cMDP"


### Track library versions and git hash of experiment

# Experiment parameters

In [4]:
config_train, config_agent, config_env, AgentClass, agent_name = prep_experiment(
        PROJECT_NAME,
        LIBRARIES_TO_TRACK,
        config_train_name="config_train.yaml",
        config_agent_name="config_agent.yaml",
        config_env_name="config_env.yaml",
    )

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: timlachner. Use `wandb login --relogin` to force relogin


INFO:root:ddopai: 0.0.7
INFO:root:mushroom_rl: 1.10.1
INFO:root:Git hash: 0272d55f1dc0d2c6c001876514a70eed15f1e0dd
INFO:root:Configuration file 'config_train.yaml' successfully loaded.
INFO:root:Configuration file 'config_agent.yaml' successfully loaded.
INFO:root:Configuration file 'config_env.yaml' successfully loaded.


In [5]:
#artifact = wandb.use_artifact('raw_data:latest')
#path = artifact.download()
#raw_data = pickle.load(open(os.path.join(path, 'raw_data.pkl'), 'rb'))

In [6]:
raw_data, val_index_start, test_index_start = get_online_data(
            config_env,
            overwrite=False
        )

## Environment parameters

* Get the environment parameters from the config file 
* Overwrite the ```lag_window```parameter with the parameter specified in the agent, if it is specified (since lag window is provided by the environment, but a tunable hyperparameter of the agent)

In [7]:
round_action = RoundAction(unit_size=config_env["unit_size"])
postprocessors = [round_action]

#ENVCLASS = get_ENVCLASS(config_env["env_class"])
#environment = set_up_env_online(ENVCLASS, raw_data, val_index_start, test_index_start, config_env, postprocessors)
environments = prepare_env_online(get_ENVCLASS=get_ENVCLASS, raw_data=raw_data, val_index_start=0, test_index_start=0, config_env=config_env, postprocessors=postprocessors)

## Agent Parameter

In [8]:
logging.info(f"Agent: {agent_name}")


if AgentClass.train_mode == "env_interaction":
    if "link" in config_agent:
        glm_link, price_function = set_up_agent(AgentClass, environments[0], config_agent)
        config_agent["g"] = glm_link
        config_agent["price_function"] = price_function
        del config_agent["link"]
    if agent_name == "Clairvoyant":
        agent = AgentClass(
        alpha=environments[0].alpha,
        beta=environments[0].beta,
        environment_info=environments[0].mdp_info,
        **config_agent
        )
    else:
        agent = AgentClass(
        environment_info=environments[0].mdp_info,
        **config_agent
        )

else:
    raise ValueError("Invalid train_mode for online training")

INFO:root:Agent: MTS


In [9]:
earlystoppinghandler = set_up_earlystoppinghandler(config_train)

In [ ]:
dataset = run_experiment(
        agent,
        environments,
        n_epochs=config_train["n_epochs"],
        n_steps=config_train["n_steps"],
        early_stopping_handler=earlystoppinghandler,
        save_best=config_train["save_best"],
        run_id=wandb.run.id,
        tracking="wandb",
        eval_step_info=False,
        print_freq=1,
        results_dir = RESULTS_DIR,
        return_dataset=True,
        return_score=False
    )

INFO:root:Starting experiment


Experiment directory: results/v9v7jgdw


  0%|          | 0/1 [00:00<?, ?it/s]/opt/anaconda3/envs/ddop/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/opt/anaconda3/envs/ddop/lib/python3.10/site-packages/statsmodels/regression/_tools.py:121: RuntimeWarning: divide by zero encountered in scalar divide
  scale = np.dot(wresid, wresid) / df_resid
INFO:root:Epoch 1: R=131262.1161618404, J=12.898472269960973
100%|██████████| 1/1 [03:30<00:00, 210.93s/it]


In [ ]:
wandb.finish()

Action,▃▁▃▂▂▂▂▄▂▂▃█▂▅▃▃▄▂▂▁▂▂▂▂▃▃▂▂▃▁▄▃▃▃▄▂▄▂▂▂
Cumulative_Reward,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
Epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Inventory,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Reward,▃▂▂▁▅▃▅▄▂▃▂▃▄▃▆▁▃▂▃▃▅▆▄▄█▆▅▅▂▆▇▁▂▂▂▃▆█▃▂
True_Cumulative_Reward,▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇████
True_Reward,▂▃▄▆▃▂▃▂▂▄▂▅▄▂▆▄▃▃▂▄▄▄▁▆▁▄▄▄▃▅▃▃▄▆█▃▃▄▄▄
t,▁▁▁▁▂▂▂▂▂▂▂▂▂▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇█
Action,3.23
Cumulative_Reward,131262.11616
Epoch,0


In [ ]:
137912.06705

137912.06705